# Causal gene regulatory network reconstruction

Cells continuously monitor their internal and external environment and calculate the amount at which each type of protein is needed. This information-processing function is carried out by [gene regulatory networks (GRNs)](https://en.wikipedia.org/wiki/Gene_regulatory_network), which control the rate of production of each protein. Two important classes of regulatory molecules in GRNs are [transcription factors](https://en.wikipedia.org/wiki/Transcription_factor) and [microRNAs](https://en.wikipedia.org/wiki/MicroRNA).

In this tutorial we will use causal inference to reconstruct GRNs from [mRNA](https://en.wikipedia.org/wiki/Messenger_RNA) and [microRNA](https://en.wikipedia.org/wiki/MicroRNA) expression data from the [GEUVADIS study](https://www.nature.com/articles/nature12531). In particular, we will use genetic variants as causal instruments and the [BioFindr software](https://github.com/tmichoel/BioFindr.jl) for model selection, as introduced in the [blessing of dimensionality notebook](2_blessing_of_dimensionality.ipynb).

## Setup the environment

In [1]:
using DrWatson
quickactivate(@__DIR__)

In [2]:
using DataFrames
using Arrow
using Gadfly
using Compose
using BioFindr

## Load data

This tutorial uses [preprocessed data files](https://github.com/lingfeiwang/findr-data-geuvadis) from the [GEUVADIS study](https://doi.org/10.1038/nature12531). We have mRNA (`t` for transcripts) and microRNA (`m`) expression data:

In [6]:
dt = DataFrame(Arrow.Table(datadir("processed","findr-data-geuvadis", "dt.arrow")));
dm = DataFrame(Arrow.Table(datadir("processed","findr-data-geuvadis", "dm.arrow")));

We also have genotype data for the strongest [eQTLs](https://en.wikipedia.org/wiki/Expression_quantitative_trait_loci) for a subset of mRNAs and microRNAs:

In [5]:
dgt = DataFrame(Arrow.Table(datadir("processed","findr-data-geuvadis", "dgt.arrow")));
dgm = DataFrame(Arrow.Table(datadir("processed","findr-data-geuvadis", "dgm.arrow")));

The [preprocessed GEUVADIS data](https://github.com/lingfeiwang/findr-data-geuvadis) has been organized such that each column of the genotype data is the strongest eQTLs for the corresponding column in the matching expression data. Usually however, eQTL mapping data will be available in the form of a table with variant IDs, gene IDs, and various eQTL associaion statistics (see the [original GEUVADIS file](https://www.ebi.ac.uk/biostudies/files/E-GEUV-1/E-GEUV-1/analysis_results/EUR373.gene.cis.FDR5.all.rs137.txt.gz) for an example). Let's artificially create such tables for our data (`p` for "pairs"):

In [9]:
dpt = DataFrame(SNP_ID = names(dgt), GENE_ID=names(dt)[1:ncol(dgt)]);
dpm = DataFrame(SNP_ID = names(dgm), GENE_ID=names(dm)[1:ncol(dgm)]);